# ImageDataset

This notebook introduces basic usages of `monai.data.ImageDataset`.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Project-MONAI/tutorials/blob/main/modules/image_dataset.ipynb)

## Setup environment

In [1]:
!python -c "import monai" || pip install -q "monai-weekly[itk, pillow]"

## Setup imports

In [2]:
# Copyright 2020 - 2021 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Copyright 2020 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
import os
import tempfile
import shutil

import nibabel as nib
import numpy as np

from monai.data import ImageDataset
from monai.transforms import Compose, EnsureChannelFirst, RandAdjustContrast, Spacing
from monai.config import print_config

print_config()

MONAI version: 0.9.1rc1+6.gcbe16eb3
Numpy version: 1.23.0
Pytorch version: 1.12.0+cu102
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: cbe16eb326830ef31479fd34ae1e44d27fc2d64d
MONAI __file__: /home/wenqil/Documents/MONAI/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.9
Nibabel version: 4.0.1
scikit-image version: 0.19.3
Pillow version: 9.2.0
Tensorboard version: 2.9.1
gdown version: 4.5.1
TorchVision version: 0.13.0+cu102
tqdm version: 4.64.0
lmdb version: 1.3.0
psutil version: 5.9.1
pandas version: 1.4.3
einops version: 0.4.1
transformers version: 4.20.1
mlflow version: 1.27.0
pynrrd version: 0.4.3

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [ ]:
# Generate some image data
tempdir = tempfile.mkdtemp()
img_ = nib.Nifti1Image(np.random.randint(0, 2, size=(20, 20, 20)), np.eye(4))
seg_ = nib.Nifti1Image(np.random.randint(0, 2, size=(20, 20, 20)), np.eye(4))
img_name = os.path.join(tempdir, "img.nii.gz")
seg_name = os.path.join(tempdir, "seg.nii.gz")
nib.save(img_, img_name)
nib.save(seg_, seg_name)
img_list = [img_name]
seg_list = [seg_name]

# Build a customised chain of transforms
`TestCompose` is used to handle the additional metadata as a part of the preprocessing pipeline.

In [4]:
class TestCompose(Compose):
    def __call__(self, data, meta):
        data = self.transforms[0](data)  # ensure channel first
        data = self.transforms[1](data)  # spacing
        meta = data.meta
        if len(self.transforms) == 3:
            return self.transforms[2](data), meta  # image contrast
        return data, meta


img_xform = TestCompose([EnsureChannelFirst(), Spacing(pixdim=(1.5, 1.5, 3.0)), RandAdjustContrast()])
seg_xform = TestCompose([EnsureChannelFirst(), Spacing(pixdim=(1.5, 1.5, 3.0), mode="nearest")])
img_dataset = ImageDataset(
    image_files=img_list,
    seg_files=seg_list,
    transform=img_xform,
    seg_transform=seg_xform,
    image_only=False,
    transform_with_metadata=True,
)
print("image shape:", img_dataset[0][0].shape)
print("seg. shape:", img_dataset[0][1].shape)

image shape: (1, 14, 14, 7)
seg. shape: (1, 14, 14, 7)


# Clean up the directory

In [5]:
shutil.rmtree(tempdir)